Yay! Exercises!

# Imports

In [92]:
import pandas as pd
#from sklearn.linear_model import LinearRegression
import numpy as np
import hashlib # for grading purposes
%matplotlib inline
from sklearn.impute import KNNImputer

### You're hired

You've been hired as the data scientist for a supermarket chain that wants to start extracting insights from their data. First, you'll start with a specific store.

Let's get our data: `store.csv` under `data/` folder

> Important: save the dataframe into the `store` variable

In [93]:
# store = ...
store = pd.read_csv('./data/store.csv')
store.head()

,date,customers
0,09-09-2013,1781
1,16-08-2015,456
2,13-09-2015,459
3,03-09-2014,1892
4,09-06-2014,1786


## Exercise 1: Index and datetime

#### 1.1) Make the index the datetime of the dates. 

In [94]:
# We expect the solution to be a dataframe

#store['date'] =
#store =
#store =            # Don't forget best practices

# YOUR CODE HERE
store['date'] = pd.to_datetime(store['date'],format='%d-%m-%Y')
store = store.set_index('date').sort_index()
store.head(10)

,customers
date,
2013-01-02,2111
2013-01-03,1833
2013-01-04,1863
2013-01-05,1509
2013-01-06,520
2013-01-07,1807
2013-01-08,1869
2013-01-09,1910
2013-01-10,1679


In [95]:
expected_hash = '660d9054fe3a6cbcfb77e2647932e3c41ff5acab9fc4d162fdc448c7c8e6ccc2'
assert hashlib.sha256(str(store.iloc[28].name).encode()).hexdigest() == expected_hash
assert hashlib.sha256(str(store.index.dtype).encode()).hexdigest() == '261738f2e43a1c47a16f043b46deb993943d61f4a2bbe5ef4b03c3fb1af362b5'


# clue: if this assert is failing, and your iloc[25].name is '2017-01-7', 
# then you are missing the "best practices" part. 
# What did we say in the Learning notebook about this? 

## Exercise 2: Time series preprocessing

#### 2.1) Let's keep granularity consistent

Sometimes datasets don't have consistent data collection, as a data scientist you should know if this is the case!

Copy `store` to a new variable called `store_complete`, **with no gap days**. You may leave the new datapoints with null values in all columns

In [96]:
# copy store to store_complete
store_complete = store.copy()

# change store_complete index so that it contains each of the days in the time range [2013-01-01, 2017-07-31]
# fill missing data with nulls
#store_complete = 

# YOUR CODE HERE
resampled_df = store_complete.resample('D').asfreq()
missing_dates = resampled_df[resampled_df.isna().all(axis=1)].index
print(missing_dates)
store_complete.resample('D')
store_complete = store_complete.resample('D').asfreq()
store_complete.index.freq

DatetimeIndex(['2013-12-25', '2014-01-01', '2014-12-25', '2015-01-01',
               '2015-07-07', '2015-12-25', '2016-01-01', '2016-01-02',
               '2016-01-03', '2016-12-25', '2017-01-01'],
              dtype='datetime64[ns]', name='date', freq=None)


<Day>

In [97]:
#### check number of nulls
assert store_complete.isnull().sum()[0] != 0, "You have 0 null values in the dataset! Remember that each missing day should correspond to a null in customers."
assert store_complete.isnull().sum()[0] == 11, "You should have found 11 days with missing data, no more no less."

#### check store_complete dataframe integrity
assert store_complete.shape[0] != 1676, "Did you fill the index with the missing days?"
assert store_complete.shape[0] == 1672, "The number of rows is not the expected."
assert store_complete.shape[1] == 1, "You shouldn't change the number of columns."
assert str(store_complete[store_complete.customers.isnull()].index[6])[:10] == '2016-01-01', "Do you have all the missing days? Is the index is ordered?"

C:\Users\sailo\AppData\Local\Temp\ipykernel_24184\1990732380.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  assert store_complete.isnull().sum()[0] != 0, "You have 0 null values in the dataset! Remember that each missing day should correspond to a null in customers."
C:\Users\sailo\AppData\Local\Temp\ipykernel_24184\1990732380.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  assert store_complete.isnull().sum()[0] == 11, "You should have found 11 days with missing data, no more no less."


## Exercise 3: Working with timestamps

#### 3.1) Worst day in 2016

What was the worst day in terms of customers in 2016?

In [98]:
# hint: the answer should be a timestamp

#worst_day_2016 =
# YOUR CODE HERE
customers_2016 = store_complete.loc['2016']
worst_day_2016 = customers_2016['customers'].idxmin()
worst_day_2016


Timestamp('2016-01-04 00:00:00')

In [99]:
expected_hash = '54ca8373016eeb8acb093f60f9d909b8fc1bcc8e37d9f762530df4053bb83a1d'
assert hashlib.sha256(str(worst_day_2016).encode()).hexdigest() == expected_hash

print(f"The worst day in 2016 was {worst_day_2016.day} of {worst_day_2016.month_name()}. Talk about new year's blues !")

The worst day in 2016 was 4 of January. Talk about new year's blues !


#### 3.2) Best Friday

Last Friday there were 3000 customers, and your boss said he's never seen such a high count of customers on a Friday. To check if your boss is correct, can you find the maximum number of customers that we've ever had on a Friday?

- _hint #1: you can use the methods at the bottom of this [page](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) directly on the index_  
- _hint #2: when operating directly on the index, you do not need to use `.dt` to use the methods_

In [118]:
#max_customers_Friday =
store_complete['Day of the Week'] = store_complete.index.weekday
friday_customers = store_complete[store_complete['Day of the Week']==4]
max_customers_Friday = friday_customers['customers'].max()
max_customers_Friday
# YOUR CODE HERE

3023.0

In [119]:
expected_hash = 'b134ce47a896876fe3111bfed26cbe06363ede8a60ada5f70fe285d04fc4b7e9'
assert hashlib.sha256(str(int(max_customers_Friday)).encode()).hexdigest() == expected_hash

print(f"Yep! The highest count we ever had on a Friday was {int(max_customers_Friday)} customers. Don't tell your boss.")

Yep! The highest count we ever had on a Friday was 3023 customers. Don't tell your boss.


## Exercise 4: Time series methods

#### 4.1) Shopping rush

A new pandemic has started, and everyone came to buy soap and isopropyl alcohol. Your boss swears to have never seen such an absolute increase in customers from one day to the next - "Yesterday there were 100 customers, today there were 5000."

To confirm if what your boss is saying is true, can you find the maximum increase in customers from one day to the next? (we want the value, not the day it happened)

In [124]:
# hint: the solution expects a float

#max_increase = 

# YOUR CODE HERE
store_complete['Customer Increase'] = store_complete.customers.diff().fillna(0)

max_increase = store_complete['Customer Increase'].max()

In [125]:
expected_hash = 'aa4b0d224e2b4488c6e3c5692347a0e26322d86dcb6bf01ab937e15d76037ee4'
assert hashlib.sha256(str(int(max_increase)).encode()).hexdigest() == expected_hash

#### 4.2) Bad month

Despite the shopping rush of the last few days, we had a bad month , with a monthly sum of customers < 45000 . What was the last month we had less than 45000 customers  (`last_bad_month`)?

In [132]:
# We expect the answer to be a monthly time period (freq='M'), so answers  
# which are of the form "the second month of the year" will not pass the grader. 
# hint: by default pandas uses freq='M'.

#sum_monthly_customers = 
#last_bad_month = 

# YOUR CODE HERE
sum_monthly_customers = store_complete['customers'].resample('M').sum()
sum_monthly_customers
#is_under_45000 = sum_monthly_customers<45000
#worst_months = is_under_45000['']

C:\Users\sailo\AppData\Local\Temp\ipykernel_24184\1815922624.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  sum_monthly_customers = store_complete['customers'].resample('M').sum()


date
2013-01-31    47471.0
2013-02-28    40587.0
2013-03-31    45299.0
2013-04-30    46740.0
2013-05-31    46286.0
2013-06-30    44876.0
2013-07-31    47874.0
2013-08-31    45789.0
2013-09-30    44540.0
2013-10-31    46424.0
2013-11-30    46783.0
2013-12-31    50110.0
2014-01-31    47417.0
2014-02-28    43819.0
2014-03-31    44311.0
2014-04-30    46035.0
2014-05-31    47225.0
2014-06-30    45680.0
2014-07-31    48253.0
2014-08-31    46066.0
2014-09-30    46414.0
2014-10-31    46535.0
2014-11-30    43037.0
2014-12-31    53532.0
2015-01-31    47302.0
2015-02-28    41473.0
2015-03-31    47794.0
2015-04-30    46139.0
2015-05-31    46838.0
2015-06-30    47870.0
2015-07-31    47190.0
2015-08-31    44656.0
2015-09-30    46182.0
2015-10-31    45835.0
2015-11-30    42896.0
2015-12-31    54431.0
2016-01-31    41572.0
2016-02-29    42653.0
2016-03-31    46244.0
2016-04-30    46771.0
2016-05-31    45385.0
2016-06-30    45609.0
2016-07-31    45258.0
2016-08-31    44915.0
2016-09-30    44899.0
2016-

In [ ]:
expected_hash = '50aa98c99f36b286c0269526800c2fc49253f75fcdef0cf02f4db4aee064ddbc'
assert hashlib.sha256(str(last_bad_month).encode()).hexdigest() == expected_hash

---